# Linear Regression from First Principles

Exploring how one of the most basic machine learning models can be implemented using PyTorch and Stochastic Gradient Descent (SGD), from first principles. We then explore the same model using the PyTorch optimiser.

## Imports

In [1]:
from typing import Sequence, Tuple

import torch
from torch.utils.data import Dataset, DataLoader

## Configuration

In [2]:
torch.manual_seed(1)

## Simple Linear Regression from First Principles

Start by creating a dataset and dataloader for the task.

In [3]:
class LinearModelData(Dataset):
    
    def __init__(self, b: float, w: float):
        self.w = torch.tensor(w)
        self.b = torch.tensor(b)
        self.X = torch.arange(-2, 2, 0.01).view(-1, 1)
        self.y = self.b + self.w * self.X + torch.randn(self.X.size())
        self.len = self.y.shape[0]
        
    def __getitem__(self, idx: float) -> Tuple[torch.FloatTensor, torch.FloatTensor]:
        return (self.X[idx], self.y[idx])
    
    def __len__(self) -> int:
        return self.len
    
    
data = LinearModelData(b=0, w=1)
print(f'n_samples = {len(data)}')
print(f'data[0] = {data[0]}')

data_loader = DataLoader(dataset=data, batch_size=5)
data_batches = list(data_loader)
print(f'mini_batch[0] = {data_batches[0]}')

n_samples = 400
data[0] = (tensor([-2.]), tensor([-3.5256]))
mini_batch[0] = [tensor([[-2.0000],
        [-1.9900],
        [-1.9800],
        [-1.9700],
        [-1.9600]]), tensor([[-3.5256],
        [-2.7402],
        [-2.6340],
        [-3.5795],
        [-2.0602]])]


Now define the model.

In [4]:
class LinearRegression(torch.nn.Module):
    """Linear regression from first principles using gradient descent."""

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.b = torch.nn.Parameter(torch.tensor([torch.randn(1)], requires_grad=True))
        self.w = torch.nn.Parameter(torch.tensor([torch.randn(1)], requires_grad=True))
        
    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        """Compute a prediction."""
        y_hat = self.b + self.w * x
        return y_hat
    
    def loss(self, y_hat: torch.FloatTensor, y: torch.FloatTensor):
        """Compute mean squared-error loss."""
        return torch.mean((y_hat - y) ** 2)
    
    def fit(
        self,
        data_loader: DataLoader,
        n_epochs: int,
        learning_rate: float
    ) -> Sequence[float]:
        """Train the model over multiple epochs recording the loss for each."""
        
        def process_batch(X: torch.FloatTensor, y: torch.FloatTensor) -> float:
            y_hat = self.forward(X)
            loss = self.loss(y_hat, y)
            loss.backward()
            
            self.w.data -= lr * self.w.grad.data
            self.w.grad.data.zero_()
            
            self.b.data -= lr * self.b.grad.data
            self.b.grad.data.zero_()
            
            return loss.detach().numpy().tolist()
            
        def process_epoch() -> float:
            return [process_batch(X, y) for X, y in data_loader][-1]
            
        lr = torch.tensor(learning_rate)
        training_run = [process_epoch() for epoch in range(n_epochs)]
        return training_run

Train the model.

In [5]:
lin_reg = LinearRegression()
print('initial parameters:')
for k, v in lin_reg.state_dict().items():
    print(f'  {k}: {v}')

print('\npost-training parameters:')
per_epoch_loss = lin_reg.fit(data_loader, n_epochs=15, learning_rate=0.05)
for k, v in lin_reg.state_dict().items():
    print(f'  {k}: {v}')

print('\nloss per-epoch:')
per_epoch_loss

initial parameters:
  b: tensor([0.5864])
  w: tensor([0.0956])

post-training parameters:
  b: tensor([0.0854])
  w: tensor([0.8475])

loss per-epoch:


[1.5875463485717773,
 1.5779364109039307,
 1.5811569690704346,
 1.580075740814209,
 1.580438494682312,
 1.5803169012069702,
 1.5803571939468384,
 1.5803438425064087,
 1.580348253250122,
 1.5803468227386475,
 1.5803474187850952,
 1.5803474187850952,
 1.5803468227386475,
 1.5803468227386475,
 1.5803474187850952]

Testing the model on unseen data.

In [6]:
test_data = LinearModelData(b=0, w=1)
rmse = torch.sqrt(torch.mean((lin_reg.forward(test_data.X) - test_data.y) ** 2))
rmse

tensor(1.0649, grad_fn=<SqrtBackward>)

Which is in-line what one would expect with a noise term that is a standard Normal distribution.

## Simple Linear Regression with the PyTorch Optimiser

In [7]:
class LinearRegressionPyTorch(torch.nn.Module):
    
    def __init__(self, input_size: int, output_size: int):
        super(LinearRegressionPyTorch, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.loss = torch.nn.MSELoss()  # this is a callable

    def forward(self, X) -> torch.FloatTensor:
        """Compute a prediction."""
        return self.linear(X)
    
    def fit(
        self,
        data_loader: DataLoader,
        n_epochs: int,
        learning_rate: float
    ) -> Sequence[float]:
        """Train the model over multiple epochs recording the loss for each."""

        def process_batch(X: torch.FloatTensor, y: torch.FloatTensor) -> float:
            y_hat = self.forward(X)
            loss = self.loss(y_hat, y)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            return loss.detach().numpy().tolist()

        def process_epoch() -> float:
            return [process_batch(X, y) for X, y in data_loader][-1]

        optimiser = torch.optim.SGD(self.parameters(), lr=0.05)
        training_run = [process_epoch() for epoch in range(n_epochs)]
        return training_run

We now training the model using `optim`.

In [8]:
lin_reg_pt = LinearRegressionPyTorch(1, 1)
lin_reg_pt.fit(data_loader, n_epochs=15, learning_rate=0.05)

[1.513684630393982,
 1.6031246185302734,
 1.5727497339248657,
 1.5829020738601685,
 1.5794906616210938,
 1.5806349515914917,
 1.580250859260559,
 1.5803797245025635,
 1.5803368091583252,
 1.5803511142730713,
 1.5803461074829102,
 1.5803478956222534,
 1.5803468227386475,
 1.5803474187850952,
 1.5803474187850952]

Testing the model on unseen data.

In [9]:
rmse = torch.sqrt(torch.mean((lin_reg_pt.forward(test_data.X) - test_data.y) ** 2))
rmse

tensor(1.0649, grad_fn=<SqrtBackward>)